<a href="https://colab.research.google.com/github/nowionlyseedaylight/2022-1-Euron-Study-Assignments/blob/Week_14/week14_nlp_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📌 week14 과제는 **13주차의 Subwords 실습**으로 구성되어 있습니다.

📌 위키독스의 딥러닝을 이용한 자연어 처리 입문 교재 실습, 관련 블로그 등의 문서 자료로 구성되어 있는 과제입니다. 

📌 안내된 링크에 맞추어 **직접 코드를 따라 치면서 (필사)** 해당 nlp task 의 기본적인 라이브러리와 메서드를 숙지해보시면 좋을 것 같습니다😊 필수라고 체크한 부분은 과제에 반드시 포함시켜주시고, 선택으로 체크한 부분은 자율적으로 스터디 하시면 됩니다.

📌 궁금한 사항은 깃허브 이슈나, 카톡방, 세션 발표 시작 이전 시간 등을 활용하여 자유롭게 공유해주세요!

In [ ]:
import nltk
# nltk colab 환경에서 실행시 필요한 코드입니다. 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

### **subwords**

👀 **내용 복습** 

* 단어 단위의 분석 - 글자 단위의 분석 - (일부는 단어 일부는 글자 단위의 분석을 진행 :  hybrid)
* 기존 word 기반 모델은 (OOV problem + 국가마다 다른 언어체계) 단점이 존재 👉  character 기반 모델 등장 배경
  * 그러나  Pure Character level model 의 수행시간이 너무 오래걸린다는 단점 존재 
  * 따라서 단어를 좀 더 쪼개서 보는 word piece 모델이 등장
* subword model 👉  word level 모델과 구조는 동일하나 더 작은 word 단위로 나누어서 보는 word pieces 를 사용하고 있는 모델 
  * BPE : subword tokenizer 알고리즘 
  * word piece :  BPE 변형 알고리즘 👉 pre-segmentation + BPE → 빈번하게 등장하는 단어들에 대해 먼저 단어사전에 추가하고 이후에 BPE 를 적용
  * sentence piece : 중국어 등 단어로 구분이 어려운 언어의 경우 raw text 에서 바로 character-level 로 나뉘어짐 
  * hybrid : 일부는 글자단위, 일부는 단어 단위로 분석 

* FastText : word2vec 을 이을 차세대 word vector learning library 로 하나의 단어에 여러 단어들이 존재하는 것으로 간주하여 학습한다.

![image](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fbu49K3%2FbtrCOKUFG7Y%2FtbffQ7RkugJ1X2kcyEYGgk%2Fimg.png)

🥰 **이하 예제를 실습하시면 됩니다.**

**1-(1)~(4) 는 필수과제, (5) 의 2개는 선택과제입니다.**


#### 🔹 1-(1) BPE

* 대표적인 subword 분리 알고리즘 
* 3) 코드실습 부분을 위주로 필사해주시면 됩니다.


📌 [Byte Pair Encoding](https://wikidocs.net/22592)







1) 기존 방식

In [ ]:
# dictionary
# 훈련 데이터에 있는 단어와 등장 빈도수
low : 5, lower : 2, newest : 6, widest : 3

# vocabulary
low, lower, newest, widest

# 테스트 단계에서 'lowest'와 같은 새로운 형태가 등장하면 oov 문제가 발생하게 됨.

2) BPE 방식

딕셔너리의 모든 단어들을 character level로 분리함.


In [ ]:
import re, collections
from IPython.display import display, Markdown, Latex

BPE 실행 횟수: 10

In [ ]:
num_merges = 10

In [ ]:
dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [ ]:
def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

In [ ]:
print(bpe_codes)

* OOV에 대처하기

In [ ]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [ ]:
encode("loki")

In [ ]:
encode("lowest")

In [ ]:
encode("lowing")

In [ ]:
encode("highing")

#### 🔹 1-(2) Subword Text

* 텐서플로우를 통해 사용할 수 있는 서브워드 토크나이저
* BPE와 유사한 알고리즘인 Wordpiece Model을 채택한 패키지
* IMDB 리뷰 토큰화, 네이버 영화 리뷰 토큰화 부분을 필사해주시면 됩니다. 

```
import tensorflow_datasets as tfds

tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_df['review'], target_vocab_size=2**13)

```


📌 [SubwordTextEncoder](https://wikidocs.net/86792)

1. IMDB 리뷰 토큰화하기

In [ ]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

train_df = pd.read_csv('IMDb_Reviews.csv')

In [ ]:
train_df['review']

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_df['review'], target_vocab_size=2**13)

In [ ]:
print(tokenizer.subwords[:100])

In [ ]:
print(train_df['review'][20])

In [ ]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_df['review'][20])))

In [ ]:
# train_df에 존재하는 문장 중 일부를 발췌
sample_string = "It's mind-blowing to me that this film was even made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

In [ ]:
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

In [ ]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

In [ ]:
# 앞서 실습한 문장에 even 뒤에 임의로 xyz 추가
sample_string = "It's mind-blowing to me that this film was evenxyz made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

In [ ]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

2. 네이버 영화 리뷰 토큰화하기

In [ ]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [ ]:
print(train_data.isnull().sum())

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_data['document'], target_vocab_size=2**13)

In [ ]:
print(tokenizer.subwords[:100])

In [ ]:
print(train_data['document'][20])

In [ ]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_data['document'][20])))

In [ ]:
sample_string = train_data['document'][21]

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

In [ ]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

#### 🔹 1-(3) SentencePiece

* ``import sentencepiece as spm`` BPE를 포함하여 기타 서브워드 토크나이징 알고리즘들을 내장한 라이브러리 
* IMDB 리뷰 토큰화, 네이버 영화 리뷰 토큰화 부분을 필사해주시면 됩니다. 


📌 [SubwordTextEncoder](https://wikidocs.net/86657)

In [ ]:
pip install sentencepiece

IMDB 리뷰 토큰화하기

In [ ]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [ ]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

In [ ]:
print('리뷰 개수 :',len(train_df)) # 리뷰 개수 출력

In [ ]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [ ]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

각 인자가 의미하는 바는 다음과 같습니다.

* input : 학습시킬 파일

* model_prefix : 만들어질 모델 이름

* vocab_size : 단어 집합의 크기

* model_type : 사용할 모델 (unigram(default), bpe, char, word)

* max_sentence_length: 문장의 최대 길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값
* user_defined_symbols: 사용자 정의 토큰

In [ ]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

In [ ]:
len(vocab_list)

In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

* encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환합니다.

* encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환합니다.

In [ ]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

In [ ]:
sp.GetPieceSize()

In [ ]:
sp.IdToPiece(430)

In [ ]:
sp.PieceToId('▁character')

In [ ]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

In [ ]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

In [ ]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

네이버 영화 리뷰 토큰화하기

In [ ]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

In [ ]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

In [ ]:
print(naver_df.isnull().values.any())

In [ ]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

In [ ]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [ ]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

In [ ]:
vocab_list.sample(10)

In [ ]:
len(vocab_list)

In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

In [ ]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

In [ ]:
sp.GetPieceSize()

In [ ]:
sp.IdToPiece(4)

In [ ]:
sp.PieceToId('영화')

In [ ]:
sp.DecodeIds([54, 200, 821, 85])

In [ ]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

In [ ]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

#### 🔹 1-(4) FastText

* 4-2) 부분을 필사해주시면 됩니다.


📌 [FastText](https://wikidocs.net/22883)


--


➕ 한국어 fastText 적용 예제 참고 블로그 (참고용으로만 봐주세요! 필수 과제 아님)
* [1](https://inahjeon.dev/fasttext/) 
* [2](https://vhrehfdl.tistory.com/77)
* [3](https://joyhong.tistory.com/137) 

**실습으로 비교하는 Word2Vec Vs. FastText**

1) Word2Vec

In [ ]:
model.wv.most_similar("electrofishing")

 이처럼 Word2Vec는 학습 데이터에 존재하지 않는 단어. 즉, 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않기 때문에 단어의 유사도를 계산할 수 없습니다.

2) FastText

In [ ]:
from gensim.models import FastText

model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar("electrofishing")

Word2Vec는 학습하지 않은 단어에 대해서 유사한 단어를 찾아내지 못 했지만, FastText는 유사한 단어를 계산해서 출력하고 있음을 볼 수 있습니다.

**한국어에서의 FastText**

#### 🔹 선택과제 


1. [Huggingface Tokenizer](https://wikidocs.net/99893)
 * 자연어 처리 스타트업 허깅페이스가 개발한 패키지 tokenizers
 * 자주 등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공
 * BERT의 WordPiece Tokenizer 실습 - 네이버 영화리뷰 


2. [Classifying Names with a Character-Level RNN](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)
 * 파이토치 실습 
 * 글자단위 RNN 모델로 18개 언어에서 유래한 수천 개의 성을 학습하고 철자를 기반으로 이름이 어떤 언어에서 유래했는지 예측해보는 문제

